Sistemas Híbridos en Biomedicina 2024-1

Prof. Fabián Vázquez & Prof. Adjunto Luis Jiménez.

***

## Proyecto Final. Fusión de Imágenes en Python.

#### López García Yalia Irais, López Ruiz Christopher, Velasco Butrón Karla Lilí & Zamora Acosta Ricardo Alexis.

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2

class PET_MRI:
    def __init__(self, master):
        self.master = master
        self.master.title("Fusión de Imágenes PET/MRI")

        marco_izquierdo = tk.Frame(master)
        marco_izquierdo.pack(side=tk.LEFT, padx=10, pady=10)

        self.imagen_pet = None
        self.imagen_mri = None
        self.imagen1 = None
        self.imagen2 = None
        self.imagen_resultado = None
        self.factor_zoom_horizontal = 1.0
        self.factor_zoom_vertical = 1.0
        self.factor_transparencia = 0.5

        self.boton_cargar_pet = tk.Button(marco_izquierdo, text="Cargar PET", command=self.cargar_imagen_pet)
        self.boton_cargar_pet.pack(pady=10)

        self.boton_cargar_mri = tk.Button(marco_izquierdo, text="Cargar MRI", command=self.cargar_imagen_mri)
        self.boton_cargar_mri.pack(pady=10)

        self.boton_corregistro = tk.Button(marco_izquierdo, text="Corregistro", command=self.registrar_imagenes)
        self.boton_corregistro.pack(pady=10)

        self.boton_fusionar = tk.Button(marco_izquierdo, text="Fusionar PET/MRI", command=self.fusionar_imagenes)
        self.boton_fusionar.pack(pady=10)

        self.boton_guardar = tk.Button(marco_izquierdo, text="Guardar PET/MRI", command=self.guardar_resultado)
        self.boton_guardar.pack(pady=10)

        self.etiqueta_transparencia = tk.Label(marco_izquierdo, text="Transparencia")
        self.etiqueta_transparencia.pack()

        self.deslizador_transparencia = ttk.Scale(marco_izquierdo, from_=0, to=1, orient=tk.HORIZONTAL, length=200,
                                                  command=self.actualizar_transparencia)
        self.deslizador_transparencia.set(0.5)
        self.deslizador_transparencia.pack(pady=10)

        self.etiqueta_zoom_horizontal = tk.Label(marco_izquierdo, text="Zoom")
        self.etiqueta_zoom_horizontal.pack()
        self.deslizador_zoom_horizontal = ttk.Scale(marco_izquierdo, from_=1, to=4, orient=tk.HORIZONTAL, length=200,
                                                     command=self.actualizar_zoom_horizontal)
        self.deslizador_zoom_horizontal.set(1)
        self.deslizador_zoom_horizontal.pack(pady=10)

        self.etiqueta_posicion_zoom_horizontal = tk.Label(marco_izquierdo, text="Posición Horizontal")
        self.etiqueta_posicion_zoom_horizontal.pack()
        self.deslizador_posicion_zoom_horizontal = ttk.Scale(marco_izquierdo, from_=0, to=1, orient=tk.HORIZONTAL,
                                                             length=200, command=self.actualizar_posicion_zoom_horizontal)
        self.deslizador_posicion_zoom_horizontal.set(0.5)
        self.deslizador_posicion_zoom_horizontal.pack(pady=10)

        self.etiqueta_posicion_zoom_vertical = tk.Label(marco_izquierdo, text="Posición Vertical")
        self.etiqueta_posicion_zoom_vertical.pack()
        self.deslizador_posicion_zoom_vertical = ttk.Scale(marco_izquierdo, from_=0, to=1, orient=tk.HORIZONTAL,
                                                           length=200, command=self.actualizar_posicion_zoom_vertical)
        self.deslizador_posicion_zoom_vertical.set(0.5)
        self.deslizador_posicion_zoom_vertical.pack(pady=10)

        marco_derecho = tk.Frame(master)
        marco_derecho.pack(side=tk.RIGHT, padx=10, pady=10)

        self.etiqueta_imagen = tk.Label(marco_derecho)
        self.etiqueta_imagen.pack()


    def cargar_imagen_pet(self):
        self.imagen_pet = filedialog.askopenfilename()
        if self.imagen_pet:
            self.imagen1 = cv2.imread(self.imagen_pet)
            self.mostrar_imagen(self.imagen1)

    def cargar_imagen_mri(self):
        self.imagen_mri = filedialog.askopenfilename()
        if self.imagen_mri:
            self.imagen2 = cv2.imread(self.imagen_mri)
            self.mostrar_imagen(self.imagen2)

    def registrar_imagenes(self):
        if self.imagen1 is not None and self.imagen2 is not None:
            self.imagen_resultado = cv2.addWeighted(self.imagen1, 0.5, self.imagen2, 0.5, 0)
            self.mostrar_imagen(self.imagen_resultado)

    def fusionar_imagenes(self):
        if self.imagen1 is not None and self.imagen2 is not None:
            self.imagen_resultado = cv2.addWeighted(self.imagen1, self.factor_transparencia,
                                                    self.imagen2, 1 - self.factor_transparencia, 0)
            self.mostrar_imagen(self.imagen_resultado)

    def guardar_resultado(self):
        if self.imagen_resultado is not None:
            ruta_archivo = filedialog.asksaveasfilename(defaultextension=".png",
                                                          filetypes=[("Archivos PNG", "*.png"),
                                                                     ("Todos los archivos", "*.*")])
            if ruta_archivo:
                cv2.imwrite(ruta_archivo, self.imagen_resultado)

    def zoom_imagen(self):
        if self.imagen_resultado is not None:
            h, w, _ = self.imagen_resultado.shape

            zoom_h = int(h / self.factor_zoom_vertical)
            zoom_w = int(w / self.factor_zoom_horizontal)

            centro_y = int(self.deslizador_posicion_zoom_vertical.get() * h)
            centro_x = int(self.deslizador_posicion_zoom_horizontal.get() * w)

            region_zoom = self.imagen_resultado[
            max(centro_y - zoom_h // 2, 0):min(centro_y + zoom_h // 2, h),
            max(centro_x - zoom_w // 2, 0):min(centro_x + zoom_w // 2, w)
            ]

            region_zoom = cv2.resize(region_zoom, (w, h))

            self.mostrar_imagen(region_zoom)


    def actualizar_zoom_horizontal(self, valor):
        self.factor_zoom_horizontal = float(valor)
        self.factor_zoom_vertical = float(valor)
        self.zoom_imagen()

    def actualizar_zoom_vertical(self, valor):
        self.factor_zoom_vertical = float(valor)
        self.factor_zoom_horizontal = float(valor)
        self.zoom_imagen()

    def actualizar_posicion_zoom_horizontal(self, valor):
        self.zoom_imagen()

    def actualizar_posicion_zoom_vertical(self, valor):
        self.zoom_imagen()

    def actualizar_transparencia(self, valor):
        self.factor_transparencia = float(valor)
        self.fusionar_imagenes()

    def mostrar_imagen(self, imagen):
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
        imagen = Image.fromarray(imagen)
        imagen = ImageTk.PhotoImage(imagen)

        self.etiqueta_imagen.config(image=imagen)
        self.etiqueta_imagen.image = imagen

if __name__ == "__main__":
    root = tk.Tk()
    app = PET_MRI(root)
    root.mainloop()